In [1]:
import cobra

def xls2model(file_path, name = 'xls_model'):
    import pandas as pd
    from cobra import Model, Reaction, Metabolite
    
    global model
    model = Model(name)
    
    # Create a dictionary that stores the metabolites information
    excel_model = pd.read_excel(file_path, sheet_name="Metabolite List")
    metabolites = {}
    
    for i in range(len(excel_model)):
        metabolites[excel_model.iloc[i,0]] = Metabolite(excel_model.iloc[i,0], 
                                             formula=excel_model.iloc[i,3], 
                                             name=excel_model.iloc[i,1], 
                                             compartment=excel_model.iloc[i,5])
    
    excel_model = pd.read_excel(file_path, sheet_name="Reaction List").fillna("") # any empty cell is read in as a NaN float object. fillna() replaces any NaN float object with an empty string
    
    biomass_reaction = []

    for i in range(len(excel_model)):
        dict_reactants = {}
        dict_products = {}
    
        reaction = Reaction(excel_model.iloc[i,0])
        reaction.name = excel_model.iloc[i,1]
        reaction.lower_bound = excel_model.iloc[i,8]
        reaction.upper_bound = excel_model.iloc[i,9]
    
        reaction_formula = excel_model.iloc[i,2]
        
        if reaction_formula.endswith("<=>" or "=>"):
            reactants = reaction_formula.strip(" <=>" or " =>")
            products = [""]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
        elif reaction_formula.startswith("<=>" or "=>"):
            reactants = [""]
            products = reaction_formula.strip("<=>" or "=>")
            if isinstance(products, list) == 0:
                products = [products]
        elif " <=>" in reaction_formula:
            reactants = reaction_formula.split("<=>")[0]
            products = reaction_formula.split("<=>")[1]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
            if isinstance(products, list) == 0:
                products = [products]
        elif " =>" in reaction_formula:
            reactants = reaction_formula.split("=>")[0]
            products = reaction_formula.split("=>")[1]
            if isinstance(reactants, list) == 0:
                reactants = [reactants]
            if isinstance(products, list) == 0:
                products = [products]
    
        if len(reactants) == 1:
            reactants = str(reactants[0])
        else:
            print("reaction formula incorrect")
    
        if len(products) == 1:
            products = str(products[0])
        else:
            print("reaction formula incorrect")
    
    
        if "+" in reactants:
            reactants = reactants.split("+")
            for j in range(len(reactants)):
                reactants[j] = reactants[j].strip()
                if " " in reactants[j]:
                    coeff_reactant_split = reactants[j].split(" ")
                    reactant_coeff = -float(coeff_reactant_split[0])
                    reactant_species = coeff_reactant_split[1]
                    dict_reactants[metabolites[reactant_species]] = reactant_coeff
                else:
                    reactant_coeff = float(-1)
                    reactant_species = reactants[j]
                    dict_reactants[metabolites[reactant_species]] = reactant_coeff
        elif len(reactants) > 1:
            reactants = reactants.strip()
            if " " in reactants:
                coeff_reactant_split = reactants.split(" ")
                reactant_coeff = -float(coeff_reactant_split[0])
                reactant_species = coeff_reactant_split[1]
                dict_reactants[metabolites[reactant_species]] = reactant_coeff
            else:
                reactant_coeff = float(-1)
                reactant_species = reactants
                dict_reactants[metabolites[reactant_species]] = reactant_coeff
            
        if "+" in products:
            products = products.split("+")
            for j in range(len(products)):
                products[j] = products[j].strip()
                if " " in products[j]:
                    coeff_product_split = products[j].split(" ")
                    product_coeff = float(coeff_product_split[0])
                    product_species = coeff_product_split[1]
                    dict_products[metabolites[product_species]] = product_coeff
                else:
                    product_coeff = float(1)
                    product_species = products[j]
                    dict_products[metabolites[product_species]] = product_coeff
        elif len(products) > 1:
            products = products.strip()
            if " " in products:
                coeff_product_split = products.split(" ")
                product_coeff = float(coeff_product_split[0])
                product_species = coeff_product_split[1]
                dict_products[metabolites[product_species]] = product_coeff
            else:
                product_coeff = float(1)
                product_species = products
                dict_products[metabolites[product_species]] = product_coeff
            
        reaction.add_metabolites({**dict_reactants, **dict_products})
        model.add_reactions([reaction])
        
        if str(excel_model.iloc[i,3]) != "":
            reaction.gene_reaction_rule = str(excel_model.iloc[i,3]) # str() is important because a balnk cell is read in as a float
    
        if excel_model.iloc[i,10] == 1:
            biomass_reaction.append(excel_model.iloc[i,0])
  
    # Now there are things in the model
    print('%i reaction' % len(model.reactions))
    print('%i metabolites' % len(model.metabolites))
    print('%i genes' % len(model.genes))
    
    if len(biomass_reaction) == 1:
        print("Objective function is:")
        print(biomass_reaction[0])
        model.objective = biomass_reaction[0]
    else:
        print("Cannot assign objective function. No or multiple reactions specified as objective functions")
        

In [2]:
xls2model(file_path = "/Users/navi/Documents/iSGOv01_CDM.xlsx", name = "iSGO")

679 reaction
548 metabolites
546 genes
Objective function is:
bio00001


In [3]:
KO_rxns_IDs = ["ac_exch",
                "ala_exch",
                "arg_exch",
                "asp_exch",
                "asn_exch",
                "cys_exch",
                "glu_exch",
                "gln_exch",
                "gly_exch",
                "his_exch",
                "ile_exch",
                "leu_exch",
                "lys_exch",
                "met_exch",
                "phe_exch",
                "pro_exch",
                "ser_exch",
                "thr_exch",
                "trp_exch",
                "tyr_exch",
                "val_exch",
                "ade_exch",
                "gua_exch",
                "ura_exch",
                "4abz_exch",
                "btn_exch",
                "fol_exch",
                "ncam_exch",
                "NADP_exch",
                "pnto_exch",
                "pydx_pydam_exch",
                "ribflv_exch",
                "thm_exch",
                "vitB12_exch",
                "FeNO3_exch",
                "MgSO4_exch",
                "MnSO4_exch",
                "CaCl2_exch",
                "NaBic_exch",
                "KPi_exch",
                "NaPi_exch"]

In [4]:
def leaveOutSim(model, KO_rxn_IDs, cutoff, LO_depth = 3):
    
    """Description: This function simulates media exclusion experiments. It returns the results of running a model when one component of media is missing (L1O: leave-one-out), two components of the media are missing (L2O), and three components of the media are missing (L3O).
    
    Usage:
    leaveOutSim(model, list_of_exchange_rxns, cutoff, LO_depth)
    
    Inputs:
    - model: cobra model to be used for the simulation
    - list_of_exchange_reactions: a list object that contains the exchange reactions for the media
    - cutoff: the fraction of the full media objective value to be considered as the cut-off for no growth. For example, if 5% of the objective value in full media is the cutoff, use 0.05
    - LO_depth: (optional) Use the integers 1, 2 or 3 to specify whether to perform a L1O only or a L1O and L2O or a L10, L2O and L3O. Default is 3.
    
    Outputs:
    Three lists:
    - L1O_growth_defect: A list of all exchange reactions that cause a growth defect in a  L1O simulation
    - L2O_growth_defect: A list of all two-way exchange reactions that cause a growth defect in a  L2O simulation
    - L3O_growth_defect: A list of all three-way exchange reactions that cause a growth defect in a  L3O simulation  """
    
    # obtain information on the objective value without any KOs
    
    solution = model.optimize()
    print("The objective value without any KOs is:", solution.objective_value, "\n")
    obj_val = solution.objective_value
    
    from cobra.flux_analysis import (
        single_gene_deletion, single_reaction_deletion, double_gene_deletion,
        double_reaction_deletion)

    # simulate single reaction KOs for the reactions listed in KO_rxn_IDs using Cobrapy's single_reaction_deletion() function
    # The function returns a dataframe that is parsed into two lists
    # list L1O_no_growth_defect contains all reactions that cause no growth defect when knocked out
    # list L1O_growth_defect contains all reactions that cause a growth defect when knocked out
    
    L1O_dataframe = single_reaction_deletion(model, KO_rxns_IDs)

    global L1O_growth_defect
    L1O_growth_defect = []
    L1O_no_growth_defect = []

    for i in range(len(L1O_dataframe)):
        rxn = str(L1O_dataframe.index[i])
        rxn = rxn.replace("frozenset({'", "")
        rxn = rxn.replace("'})", "")
        if L1O_dataframe["growth"][i] < cutoff*obj_val:
            L1O_growth_defect.append(rxn)
        else:
            L1O_no_growth_defect.append(rxn)

    print("L1O exclusions that lead to growth defect:") 
    for component in L1O_growth_defect:
        print(component)

    print("\n")
    
    if LO_depth == 1:
        return # terminate here if the user wants L1O only
    
    # simulate double reaction KOs for the reactions listed in KO_rxn_IDs using Cobrapy's double_reaction_deletion() function
    # The function returns a dataframe that is parsed into two lists just above
    
    L2O_dataframe = double_reaction_deletion(model, L1O_no_growth_defect)

    global L2O_growth_defect
    L2O_growth_defect = []
    L2O_no_growth_defect = []

    for j in range(len(L2O_dataframe)):
        double_rxn = str(L2O_dataframe.index[j])
        double_rxn = double_rxn.replace("frozenset({'", "")
        double_rxn = double_rxn.replace("'})", "")
        double_rxn = double_rxn.replace("', '", " + ")
        if L2O_dataframe["growth"][j] < cutoff*obj_val:
            L2O_growth_defect.append(double_rxn)
        else:
            L2O_no_growth_defect.append(double_rxn)

    print("L2O exclusions that lead to growth defect:") 
    for components in L2O_growth_defect:
        print(components)

    print("\n")
    
    if LO_depth == 2:
        return
    
    # Generate a list for the L3O reactions. Each list contains 3 reactions. 
    
    L3O_rxns = []

    for rxns in L2O_no_growth_defect:
        double_rxn_list = rxns.split(" + ")
        if len(double_rxn_list) == 2:
            for rxn in L1O_no_growth_defect:
                if rxn not in double_rxn_list:
                    triple_rxn_list = double_rxn_list + [rxn]
                    triple_rxn_list.sort()
                    if triple_rxn_list not in L3O_rxns:
                        L3O_rxns.append(triple_rxn_list)
                    triple_rxn_list = []
        double_rxn_list = []
    
    # Remove redundant lists from the list of lists L3O_rxns. 
    # Also remove lists that contain L2O_growth_defect reactions.
    
    for rxns in L2O_growth_defect:
        double_rxn_list = rxns.split(" + ")
        if len(double_rxn_list) == 2:
            for rxns in L3O_rxns:
                if double_rxn_list[0] in rxns and double_rxn_list[1] in rxns:
                        L3O_rxns.remove(rxns)
        double_rxn_list = []
    
    global L3O_growth_defect
    L3O_growth_defect = []
    
    # simulate KOs for each list of three reactions in the list of lists L3O_rxns
    
    for rxns in L3O_rxns:
        with model:
            for i in range(len(rxns)):
                model.reactions.get_by_id(rxns[i]).knock_out()
            solution = model.optimize()
            if solution.objective_value < cutoff*obj_val: 
                L3O_growth_defect.append(rxns)
    
    print("L3O exclusions that lead to growth defect:") 
    for components in L3O_growth_defect:
        L3O_item = ' + '.join(components)
        print(L3O_item)
    
    if len(L3O_growth_defect) == 0:
        print("none")
                        

In [5]:
leaveOutSim(model, KO_rxns_IDs, 0.07)

The objective value without any KOs is: 26.693130651762296 

L1O exclusions that lead to growth defect:
thm_exch
pydx_pydam_exch
MgSO4_exch
ribflv_exch
pnto_exch


L2O exclusions that lead to growth defect:
KPi_exch + NaPi_exch
ncam_exch + NADP_exch


L3O exclusions that lead to growth defect:
none
